In [1]:
# import libraries needed
import IPython
import pandas as pd
import numpy as np
from statistics import mode
import matplotlib.pyplot as plt  
from sklearn.preprocessing import MinMaxScaler
from IPython.display import display, HTML
from pprint import pprint
import xgboost as xgb
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from scipy import stats
import os

In [2]:
df = pd.read_csv('./data/diabetic_data.csv').sort_values(by=['patient_nbr'])
len(df)

101766

In [3]:
# drop rows

mask = df['gender'] != 'Unknown/Invalid'
mask &= df['diag_1'] != '?'  # TODO: only consider primary diag
# mask &= df['medical_specialty'] != '?'
df = df[mask]
print(len(df))

# consider only the first encounter for each patient
df = df.drop_duplicates(subset='patient_nbr', keep='first')

# remove all encounters that resulted in either discharge to a hospice or patient death
# to avoid biasing our analysis
df = df[~df['discharge_disposition_id'].isin([11, 13, 14, 19, 20, 21])]

print(len(df))

101742
69650


# TODO
1. use top10 medical_specialty 
2. use diag 2, 3
3. use 24 features
6. check merge id

In [4]:
# # drop columns

# drop unrelated columns
df = df.drop(columns=["encounter_id", "patient_nbr"])  

# drop columns with too many missing values
df = df.drop(columns=["payer_code", "weight", "medical_specialty"]) 

# drop columns having same value in each row
df = df.drop(columns=["citoglipton", "examide"])

# drop diag_2, diag_3
df = df.drop(columns=["diag_2", "diag_3"])
df = df.drop(columns=["metformin",
    "repaglinide",
    "nateglinide",
    "chlorpropamide",
    "glimepiride",
    "acetohexamide",
    "glipizide",
    "glyburide",
    "tolbutamide",
    "pioglitazone",
    "rosiglitazone",
    "acarbose",
    "miglitol",
    "troglitazone",
    "tolazamide",
    "insulin",
    "glyburide-metformin",
    "glipizide-metformin",
    "glimepiride-pioglitazone",
    "metformin-rosiglitazone",
    "metformin-pioglitazone"
])

In [5]:
num_col = [
    'time_in_hospital',
    'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient',
    'number_emergency', 'number_inpatient', 'number_diagnoses',
]

# Removing skewnewss and kurtosis using log transformation if it is above a threshold value (2)

statdataframe = pd.DataFrame()
statdataframe['numeric_column'] = num_col
skew_before = []
skew_after = []

kurt_before = []
kurt_after = []

standard_deviation_before = []
standard_deviation_after = []

log_transform_needed = []

log_type = []

for i in num_col:
    skewval = df[i].skew()
    skew_before.append(skewval)
    
    kurtval = df[i].kurtosis()
    kurt_before.append(kurtval)
    
    sdval = df[i].std()
    standard_deviation_before.append(sdval)
    
    if (abs(skewval) >2) & (abs(kurtval) >2):
        log_transform_needed.append('Yes')
        
        if len(df[df[i] == 0])/len(df) <=0.02:
            log_type.append('log')
            skewvalnew = np.log(pd.DataFrame(df[train_data[i] > 0])[i]).skew()
            skew_after.append(skewvalnew)
            
            kurtvalnew = np.log(pd.DataFrame(df[train_data[i] > 0])[i]).kurtosis()
            kurt_after.append(kurtvalnew)
            
            sdvalnew = np.log(pd.DataFrame(df[train_data[i] > 0])[i]).std()
            standard_deviation_after.append(sdvalnew)
            
        else:
            log_type.append('log1p')
            skewvalnew = np.log1p(pd.DataFrame(df[df[i] >= 0])[i]).skew()
            skew_after.append(skewvalnew)
        
            kurtvalnew = np.log1p(pd.DataFrame(df[df[i] >= 0])[i]).kurtosis()
            kurt_after.append(kurtvalnew)
            
            sdvalnew = np.log1p(pd.DataFrame(df[df[i] >= 0])[i]).std()
            standard_deviation_after.append(sdvalnew)
            
    else:
        log_type.append('NA')
        log_transform_needed.append('No')
        
        skew_after.append(skewval)
        kurt_after.append(kurtval)
        standard_deviation_after.append(sdval)

statdataframe['skew_before'] = skew_before
statdataframe['kurtosis_before'] = kurt_before
statdataframe['standard_deviation_before'] = standard_deviation_before
statdataframe['log_transform_needed'] = log_transform_needed
statdataframe['log_type'] = log_type
statdataframe['skew_after'] = skew_after
statdataframe['kurtosis_after'] = kurt_after
statdataframe['standard_deviation_after'] = standard_deviation_after

display(statdataframe)

# performing the log transformation for the columns determined to be needing it above.

for i in range(len(statdataframe)):
    if statdataframe['log_transform_needed'][i] == 'Yes':
        colname = str(statdataframe['numeric_column'][i])
        
        if statdataframe['log_type'][i] == 'log':
            df = df[df[colname] > 0]
            df[colname + "_log"] = np.log(df[colname])
            
        elif statdataframe['log_type'][i] == 'log1p':
            df = df[df[colname] >= 0]
            df[colname + "_log1p"] = np.log1p(df[colname])

,numeric_column,skew_before,kurtosis_before,standard_deviation_before,log_transform_needed,log_type,skew_after,kurtosis_after,standard_deviation_after
0,time_in_hospital,1.179887,1.010742,2.940580,No,NA,1.179887,1.010742,2.940580
1,num_lab_procedures,-0.222364,-0.274412,19.795305,No,NA,-0.222364,-0.274412,19.795305
2,num_procedures,1.252248,0.647099,1.740257,No,NA,1.252248,0.647099,1.740257
3,num_medications,1.408004,3.696858,8.272745,No,NA,1.408004,3.696858,8.272745
4,number_outpatient,9.368730,166.938568,1.133894,Yes,log1p,3.030485,9.776890,0.395807
5,number_emergency,18.382291,799.856970,0.593881,Yes,log1p,4.109939,20.347462,0.249621
6,number_inpatient,4.356649,32.342584,0.829761,Yes,log1p,2.090042,3.967241,0.389766
7,number_diagnoses,-0.751250,-0.318635,1.993277,No,NA,-0.751250,-0.318635,1.993277


In [6]:
# merge ids with same meaning
def merge(df, col, same_ids):
    for ids in same_ids:
        for k in ids[1:]:
            df[col] = df[col].replace(k, ids[0])
    return df

df = merge(df, 'admission_type_id', [
    [1, 2, 7],  # emergence
    [5, 6, 8],  # not avaliable
])
df = merge(df, 'discharge_disposition_id', [
    [18, 25, 26],  # not avaliable
    [1, 6, 8],  # to home
    [2, 3, 4, 5],  # discharge to another hospital
    [10, 12, 15, 16, 17],  # discharge to outpatient
])
df = merge(df, 'admission_source_id', [
    [1, 2, 3], # Referral
    [4, 5, 6, 10, 22, 25], # from another hospital
    [9, 15, 17, 20, 21]  # not avaliable
])

In [7]:
# convert diagnosis information: reference from original paper
def in_range(x, bounds=None, sets=None):
    if bounds is None:
        bounds = []
    elif not isinstance(bounds[0], tuple):
        bounds = [bounds]
    
    if sets is None:
        sets = []
    elif not isinstance(sets, tuple):
        sets = [sets]
        
    for (l, r) in bounds:
        if x >= l and x < r:
            return True
    return x in sets

def convert_diag_func(x):
    ranges = {
        ((390, 460), 785),
        ((460, 520), 786),
        ((520, 580), 787),
        (None, 250),
        ((800, 1000), None),
        ((710, 740), None),
        ((580, 630), 788),
        ((140, 240), None),
    }
    if x == '?':
        return -1
    elif isinstance(x, str) and (x.startswith('V') or x.startswith('E')):
        return 0

    for i, (bounds, sets) in enumerate(ranges):
        if in_range(int(float(x)), bounds, sets):
            return i+1
    else:
        return 0
        
df['diag_1'] = df['diag_1'].apply(convert_diag_func)
# df['diag_2'] = df['diag_2'].apply(convert_diag_func)
# df['diag_3'] = df['diag_3'].apply(convert_diag_func)

In [8]:
# df = df[df['diag_1']  == 4]

In [9]:
def show_crosstab(col):
    grouped = df.groupby([col, 'readmitted']).size().to_frame().reset_index()
    for c, n in df.groupby([col]).size().iteritems():
        grouped.loc[grouped[col] == c, 0] /= n
    grouped[0] = grouped[0].apply(lambda x: "{:.2%}".format(x))
    crosstab = pd.crosstab(grouped[col], grouped['readmitted'], values=grouped[0], aggfunc=lambda x: x)
    display(crosstab)

show_crosstab('max_glu_serum')
show_crosstab('A1Cresult')
# show_crosstab('medical_specialty')

readmitted,<30,>30,NO
max_glu_serum,,,
>200,7.68%,29.03%,63.29%
>300,8.30%,31.30%,60.41%
None,7.23%,26.65%,66.12%
Norm,7.34%,25.25%,67.41%


readmitted,<30,>30,NO
A1Cresult,,,
>7,6.55%,25.26%,68.19%
>8,6.37%,25.96%,67.67%
None,7.40%,27.01%,65.59%
Norm,6.86%,24.07%,69.08%


In [10]:
def encode(df, col, d):
    df[col] = df[col].apply(lambda x: d[x])
    return df

def encode_with_none(df, col, d):
    df[col+'_is_none'] = df[col] == 'None'
    d['None'] = -1
    return encode(df, col, d)

df['age'] = df['age'].apply(lambda x: int(x[1]) * 10 + 5)  # encode [0-10) - [90, 100] to 0 - 9
df = encode(df, 'readmitted', {"NO": 0, ">30": 1, "<30": 2})
df = encode_with_none(df, 'max_glu_serum', {">300": 2, ">200": 1, "Norm": 0})
df = encode_with_none(df, 'A1Cresult', {">8": 2, ">7": 1, "Norm": 0})

df = pd.get_dummies(df, drop_first = False, columns=[
    'race', 
    'gender', 
    'admission_type_id', 
    'discharge_disposition_id',
    'admission_source_id', 
    'diag_1', # 'diag_2', 'diag_3',
    'change', 
    'diabetesMed',
#     'medical_specialty',
])

In [11]:
df['number_outpatient_log1p']

4267      0.000000
5827      0.000000
67608     0.000000
17494     0.000000
2270      0.000000
14180     0.000000
18234     0.000000
15848     0.000000
61382     1.386294
2279      0.000000
7866      0.000000
25911     0.000000
1083      0.000000
2001      0.000000
11049     0.000000
2484      0.000000
17342     0.000000
23541     0.000000
4407      0.000000
7038      0.000000
2005      0.000000
10001     0.000000
21483     0.000000
3294      0.000000
22342     0.000000
36317     0.000000
4333      0.000000
18558     0.000000
36720     0.000000
18390     0.000000
            ...   
101755    0.000000
99544     0.000000
96047     0.000000
97982     0.000000
96274     0.000000
98897     0.000000
99798     0.000000
99556     0.000000
91913     0.000000
91774     0.000000
93108     0.000000
93052     0.000000
93050     0.000000
96345     0.000000
93742     0.000000
95669     0.000000
100090    0.693147
90717     0.000000
92165     0.000000
95032     0.000000
94231     0.000000
90933     0.

In [12]:
df.head(18).T

,4267,5827,67608,17494,2270,14180,18234,15848,61382,2279,7866,25911,1083,2001,11049,2484,17342,23541
age,55,55,85,85,35,65,65,45,75,75,65,75,65,75,85,65,45,85
time_in_hospital,8,2,4,3,5,10,9,2,14,12,8,1,2,7,7,4,1,7
num_lab_procedures,77,49,68,46,49,54,52,50,21,47,57,31,15,27,77,47,35,51
num_procedures,6,1,2,0,0,2,1,5,0,2,6,1,0,3,0,4,5,0
num_medications,33,11,23,20,5,19,16,13,15,18,31,9,14,16,12,16,13,13
number_outpatient,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0
number_emergency,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
number_inpatient,0,0,0,0,0,1,0,0,2,0,0,0,0,0,0,0,0,1
number_diagnoses,8,3,9,9,3,9,9,9,7,9,9,7,9,9,9,7,8,9
max_glu_serum,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


# Train and test

In [13]:
y = df['readmitted'].values # .astype(int)
X = df.drop(columns='readmitted').values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

X_train, y_train = SMOTE(random_state=20).fit_sample(X_train, y_train)

weight_per_class = np.sqrt(y.shape[0] / (df['readmitted'].value_counts() * 3))
w_train = pd.Series(y_train).map(weight_per_class).values

print(pd.Series(y_train).value_counts())
print(pd.Series(y_test).value_counts())

xg_train = xgb.DMatrix(X_train, label=y_train, weight=w_train)
# xg_train = xgb.DMatrix(X_train, label=y_train)
xg_test = xgb.DMatrix(X_test, label=y_test)

def f1_macro(preds, dtrain):
    labels = dtrain.get_label()
    return "f1_macro", f1_score(labels, preds, average='macro')

os.environ["CUDA_VISIBLE_DEVICES"] = "4"
param = {
    'booster': 'dart',
    "objective": 'multi:softmax',
    "num_class": 3,
    "silent": 1,
    
    "gpu_id": 0,
    "tree_method": "gpu_hist",
    
    'lambda': 1,
    'alpha': 0,
    
    "eta": 1,
    "max_depth": 6,
}

num_round = 6

watchlist = [(xg_train, 'train'), (xg_test, 'test')]
bst = xgb.train(param, xg_train, num_round, watchlist, feval=f1_macro)
# test
y_pred = bst.predict(xg_test)

print(classification_report(y_pred, y_test))
display(pd.crosstab(pd.Series(y_pred, name='y_pred'), 
                  pd.Series(y_test, name='y_gt'), 
                  margins=True))
# xgb.cv(param, xg_train, num_round, nfold=3, feval=f1_macro)

2    36765
1    36765
0    36765
dtype: int64
0    9243
1    3703
2     984
dtype: int64
[0]	train-merror:0.371765	test-merror:0.497703	train-f1_macro:0.519776	test-f1_macro:0.38674
[1]	train-merror:0.342093	test-merror:0.442642	train-f1_macro:0.558673	test-f1_macro:0.416983
[2]	train-merror:0.328912	test-merror:0.423187	train-f1_macro:0.582039	test-f1_macro:0.423235
[3]	train-merror:0.320008	test-merror:0.412922	train-f1_macro:0.599621	test-f1_macro:0.43099
[4]	train-merror:0.313956	test-merror:0.392391	train-f1_macro:0.611148	test-f1_macro:0.436157
[5]	train-merror:0.30657	test-merror:0.389734	train-f1_macro:0.621678	test-f1_macro:0.436864
              precision    recall  f1-score   support

         0.0       0.75      0.73      0.74      9423
         1.0       0.38      0.39      0.39      3585
         2.0       0.18      0.19      0.18       922

   micro avg       0.61      0.61      0.61     13930
   macro avg       0.44      0.44      0.44     13930
weighted avg       0.62 

y_gt,0,1,2,All
y_pred,,,,
0.0,6922,1986,515,9423
1.0,1887,1404,294,3585
2.0,434,313,175,922
All,9243,3703,984,13930
